In [1]:
import numpy as np
import pandas as pd
import cv2
import torch
from sklearn import preprocessing
import torchvision.models as models
from torchsummary import summary
import torch.nn as nn
from src.models import WhaleDoModel
from sklearn.model_selection import StratifiedKFold, train_test_split
from src.utils import *
from torchvision import transforms
import matplotlib.pyplot as plt
import cv2
import torch
import imutils
import tensorflow as tf
from src.dataloader import WhaleDoDataset
import pickle as pkl
from pytorch_metric_learning import losses, miners
from torch.utils.data import DataLoader
from src.config import get_config



2022-06-10 17:16:09.057995: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
config = get_config()
# load and parse the dataframe and get the label encoder as well
train_df, label_encoder = load_csv_and_parse_dataframe(config['csv_name'], root_dir=config['root_dir'], drop_columns=['timestamp', 'encounter_id'])
# get the average height and width of the dataset (rotate if viewpoint -1 or 1 (left / right))
config['dataset']['height'], config['dataset']['width'] = get_avg_height_width(train_df)
# get the mean and std of the dataset
config['dataset']['mean'], config['dataset']['std'] = get_mean_and_std_of_dataset(train_df)


# create the dataset objects
train_data = WhaleDoDataset(train_df, config, mode='train')

# create dataloaders
train_loader = DataLoader(train_data, config['batch_size'], shuffle=True)

# init loss function and a miner. The miner samples for training samples
loss_func = losses.TripletMarginLoss(margin=config['margin'])
miner = miners.TripletMarginMiner(margin=config['margin'])


device = config['device']
# init model and optimizer
model = WhaleDoModel(config)
optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'])
model.to(device)


/home/ando/.pyenv/versions/3.10.0/envs/whaledoenv/lib/python3.10/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


WhaleDoModel(
  (backbone): BackBone(
    (model): ResNet(
      (conv1): Conv2d(4, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inp

In [4]:
for i, batch in enumerate(train_loader):

    #move tensors to device (gpu or cpu)
    x_batch, y_batch = batch['image'].to(config['device']), batch['label'].to(config['device'])

    #set the gradients to zero
    optimizer.zero_grad()

    #compute embeddings
    embeddings = model(x_batch)

    #mine for hard pairs
    # mined_pairs = miner(embeddings, y_batch)
    #compute loss
    loss = loss_func(embeddings, y_batch)#, mined_pairs)

    #calculate gradients
    loss.backward()
    #update weights
    optimizer.step()

    break

In [5]:
for i, (x_batch, y_batch) in enumerate(train_loader):
    optimizer.zero_grad()
    embeddings = model(x_batch)
    loss = loss_func(embeddings, y_batch)
    loss.backward()
    optimizer.step()

    print(i)
    break

0
